# Redes Sociais - APS 1


Uma rede de inclusão de arquivo de código-fonte Linux (v3.16). Os vértices representam arquivos fonte e uma aresta direcionada indica se um arquivo inclui outro.

_Nota: os grafos são dirigidos, conforme informado no agregador de databases [Netzschleuder](https://networks.skewed.de/), do qual foi extraído a base utilizada aqui._

## Pré-requisitos

In [1]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw

In [2]:
PATH = 'edges.csv'

## Verificação dos dados importados

O arquivo relevante para nossa análise é `edges.csv`, que representa uma lista de valores separados por _vírgulas_, com uma aresta por linha. O significado de cada coluna do arquivo é:

- Primeira coluna: ID do nó de origem (_source_)

- Segunda coluna: ID do nó de destino (_target_)

Agora, devemos inspecionar os valores armazenados em `edges.csv` (vamos imprimir as 5 primeiras linhas do arquivo, para não sobrecarregar este notebook):

In [3]:
with open(PATH) as file:
    
    # Ignoramos o cabeçalho
    next(file)

    i = 0
    for line in file:
        parts = line.split(',')
        print(parts[0], parts[1])
        i += 1
        if i == 5:
            break

0 9

0 10

0 11

0 12

0 13



## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [4]:
g = gte.Graph(directed=True) # pois o grafo é direcionado, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [5]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

In [6]:
def get_or_add_edge(g, id_origin, id_destiny):
    e = g.edge_by_ids(id_origin, id_destiny)
    if e is None:
        e = g.add_edge_by_ids(id_origin, id_destiny)
    return e

Depois de criar o novo grafo, vamos armazenar os valores de `out.linux` nele, o transformando em uma rede.

In [7]:
with open(PATH) as file:
    
    # Ignoramos o cabeçalho
    next(file)

    i = 1
    for line in file:
        parts = line.split(',')

        # Define os IDs de origem e destino.
        id_origin = int(parts[0])
        id_destiny = int(parts[1])
        
        # Adiciona os vértices.
        get_or_add_vertex(g, id_origin)
        get_or_add_vertex(g, id_destiny)

        # Adiciona a aresta correspondente a esta linha.
        get_or_add_edge(g, id_origin, id_destiny)
        
        i += 1

print(f'Foram lidas {i} linhas.') 

Foram lidas 213955 linhas.


## Reposicionamento dos vértices

A função `draw.sfdp_layout` será usada para direcionar a disposição dos vértices de forma a destacar padrões de agrupamento, a partir do algoritmo Scalable Force-Directed Placement.
Esse algoritmo simula forças a fim de mover os vértices para posições onde as forças estão equilibradas.

In [8]:
layout = draw.sfdp_layout(g)

In [9]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [10]:
g = gte.clean(g)

In [11]:
gte.save(g, 'linux.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [12]:
r = netpixi.render('linux.net.gz', infinite = True)

Por fim, devemos ajustar a visualização da renderização.

In [13]:
r.vertex_default(size=4, bwidth=1)

In [14]:
r.edge_default(width=1)